# **_Reinforcement Learning tools for Auto-Stock Trading_**  

### 1. Importing Necessary Libraries

In [1]:
#Basic Data Science Libraries
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')
%matplotlib inline

In [2]:
#Finrl utilities
from finrl import config
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import data_split
from finrl.agents.stablebaselines3.models import DRLAgent

C:\Users\Vansh\anaconda3\envs\tf_gpu\lib\site-packages\exchange_calendars\exchange_calendar.py:2347: FutureWarning: 'T' is deprecated and will be removed in a future version. Please use 'min' instead of 'T'.
  align: pd.Timedelta | str = pd.Timedelta(1, "T"),


In [3]:
#Processing Utilities
import datetime
import itertools


In [4]:
#Make finrl imports accessible
import sys
sys.path.append("../FinRL-Library")

In [5]:
#Setup libraries
from __future__ import annotations
#postponed evaluation of type annotations and evaluation available at runtime

In [6]:
#other imports will be used wherever applicable

In [7]:
#Symbols of BSE SENSEX30 whose data is to be downloaded
# symbols = [
#     'AXISBANK.BO', 'BAJAJ-AUTO.BO', 'BAJFINANCE.BO', 'BAJAJFINSV.BO',
#     'BHARTIARTL.BO', 'DRREDDY.BO', 'HCLTECH.BO', 'JSWSTEEL.BO', 'HDFCBANK.BO',
#     'HINDUNILVR.BO', 'ICICIBANK.BO', 'INDUSINDBK.BO', 'INFY.BO', 'ITC.BO',
#     'KOTAKBANK.BO', 'LT.BO', 'M&M.BO', 'MARUTI.BO', 'NESTLEIND.BO',
#     'NTPC.BO', 'ONGC.BO', 'POWERGRID.BO', 'RELIANCE.BO', 'SBIN.BO',
#     'SUNPHARMA.BO', 'TCS.BO', 'TECHM.BO', 'TITAN.BO', 'ULTRACEMCO.BO','ASIANPAINT.BO'
# ]


In [8]:
#Globally accesible training and trading s/e
TRAIN_START_DATE = '2010-01-01'
TRAIN_END_DATE = '2020-07-01'
TRADE_START_DATE = '2020-07-01'
TRADE_END_DATE = '2023-05-01'

In [9]:
#How we downloaded the data
# df_raw = YahooDownloader(start_date = TRAIN_START_DATE,
#                                 end_date = TRADE_END_DATE,
#                                 ticker_list = symbols).fetch_data()

### 2. Loading the Data

In [10]:
df_raw=pd.read_csv('datasets/BSE30.csv')

In [11]:
df_raw.head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,90.750000,90.750000,88.550003,48.861801,19140,ASIANPAINT.BO,4
1,2009-01-02,105.800003,109.599998,103.459999,71.914917,4536215,AXISBANK.BO,4
2,2009-01-02,206.050003,210.500000,196.500000,158.413025,52648,BAJAJ-AUTO.BO,4
3,2009-01-02,15.140000,15.800000,14.975000,13.401811,136590,BAJAJFINSV.BO,4
4,2009-01-02,6.660000,6.970000,6.350000,2.746401,274220,BAJFINANCE.BO,4


In [12]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105703 entries, 0 to 105702
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   date    105703 non-null  object 
 1   open    105703 non-null  float64
 2   high    105703 non-null  float64
 3   low     105703 non-null  float64
 4   close   105703 non-null  float64
 5   volume  105703 non-null  int64  
 6   tic     105703 non-null  object 
 7   day     105703 non-null  int64  
dtypes: float64(4), int64(2), object(2)
memory usage: 6.5+ MB


### 3. Preprocessing the data

In [13]:
from finrl.config import INDICATORS
from dataprocessing import FeatureEngineer, load_dataset, data_split, convert_to_datetime

fe = FeatureEngineer(use_technical_indicator=True,
                      tech_indicator_list = INDICATORS,
                      use_vix=False,
                      use_turbulence=True,
                      user_defined_feature = False)

processed = fe.preprocess_data(df_raw)

Successfully added technical indicators


C:\Users\Vansh\Downloads\BTP-2-4-24\dataprocessing.py:236: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_price_pivot = df_price_pivot.pct_change()


Successfully added turbulence index


In [14]:
processed

,date,open,high,low,close,volume,tic,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,90.750000,90.750000,88.550003,48.861801,19140,ASIANPAINT.BO,4,0.000000,50.523346,48.068260,100.000000,66.666667,100.000000,48.861801,48.861801,0.000000
1,2009-01-02,105.800003,109.599998,103.459999,71.914917,4536215,AXISBANK.BO,4,0.000000,50.523346,48.068260,100.000000,66.666667,100.000000,71.914917,71.914917,0.000000
2,2009-01-02,206.050003,210.500000,196.500000,158.413025,52648,BAJAJ-AUTO.BO,4,0.000000,50.523346,48.068260,100.000000,66.666667,100.000000,158.413025,158.413025,0.000000
3,2009-01-02,15.140000,15.800000,14.975000,13.401811,136590,BAJAJFINSV.BO,4,0.000000,50.523346,48.068260,100.000000,66.666667,100.000000,13.401811,13.401811,0.000000
4,2009-01-02,6.660000,6.970000,6.350000,2.746401,274220,BAJFINANCE.BO,4,0.000000,50.523346,48.068260,100.000000,66.666667,100.000000,2.746401,2.746401,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105698,2023-04-28,981.000000,992.500000,979.250000,986.799988,26056,SUNPHARMA.BO,4,-0.263414,1019.314408,965.265603,50.085294,14.481255,1.567920,983.446670,985.046100,43.069415
105699,2023-04-28,3208.000000,3227.199951,3197.149902,3175.769043,51644,TCS.BO,4,-15.398183,3235.633708,3045.249324,48.649310,67.966063,0.407494,3131.238102,3257.234477,43.069415
105700,2023-04-28,983.000000,1026.650024,982.950012,986.955139,279514,TECHM.BO,4,-22.941437,1102.074200,929.293964,44.970681,-99.119890,22.233939,1033.226742,1032.633037,43.069415
105701,2023-04-28,2663.500000,2679.300049,2620.050049,2640.399902,32742,TITAN.BO,4,43.161331,2669.834479,2492.325506,60.306098,116.653875,37.463255,2542.391650,2482.099988,43.069415


In [15]:
df=processed

In [16]:
list_ticker = df["tic"].unique().tolist()
# only apply to daily level data, need to fix for minute level
list_date = list(pd.date_range(df['date'].min(),df['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

df_full = pd.DataFrame(combination,columns=["date","tic"]).merge(df,on=["date","tic"],how="left")
df_full = df_full[df_full['date'].isin(df['date'])]
df_full = df_full.sort_values(['date','tic'])
df_full = df_full.fillna(1)

In [17]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Index: 105900 entries, 0 to 156899
Data columns (total 17 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   date          105900 non-null  object 
 1   tic           105900 non-null  object 
 2   open          105900 non-null  float64
 3   high          105900 non-null  float64
 4   low           105900 non-null  float64
 5   close         105900 non-null  float64
 6   volume        105900 non-null  float64
 7   day           105900 non-null  float64
 8   macd          105900 non-null  float64
 9   boll_ub       105900 non-null  float64
 10  boll_lb       105900 non-null  float64
 11  rsi_30        105900 non-null  float64
 12  cci_30        105900 non-null  float64
 13  dx_30         105900 non-null  float64
 14  close_30_sma  105900 non-null  float64
 15  close_60_sma  105900 non-null  float64
 16  turbulence    105900 non-null  float64
dtypes: float64(15), object(2)
memory usage: 14.5+ MB


In [18]:
df=df_full

In [19]:
df.head()

,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,turbulence
0,2009-01-02,ASIANPAINT.BO,90.750000,90.750000,88.550003,48.861801,19140.0,4.0,0.0,50.523346,48.06826,100.0,66.666667,100.0,48.861801,48.861801,0.0
1,2009-01-02,AXISBANK.BO,105.800003,109.599998,103.459999,71.914917,4536215.0,4.0,0.0,50.523346,48.06826,100.0,66.666667,100.0,71.914917,71.914917,0.0
2,2009-01-02,BAJAJ-AUTO.BO,206.050003,210.500000,196.500000,158.413025,52648.0,4.0,0.0,50.523346,48.06826,100.0,66.666667,100.0,158.413025,158.413025,0.0
3,2009-01-02,BAJAJFINSV.BO,15.140000,15.800000,14.975000,13.401811,136590.0,4.0,0.0,50.523346,48.06826,100.0,66.666667,100.0,13.401811,13.401811,0.0
4,2009-01-02,BAJFINANCE.BO,6.660000,6.970000,6.350000,2.746401,274220.0,4.0,0.0,50.523346,48.06826,100.0,66.666667,100.0,2.746401,2.746401,0.0


In [20]:
df.shape

(105900, 17)

### 4.Splitting Training and Trading Data

In [21]:
train = data_split(df, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(df, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))
     

77550
21120


In [23]:
train.to_csv('train_data.csv')
trade.to_csv('trade_data.csv')

### 5. Construction of Trading Environment

In [24]:
from TradingEnv import StockTradingEnv

In [26]:


# Set the corresponding values to 'True' for the algorithms that you want to use
if_using_a2c = True
if_using_ddpg = True
if_using_ppo = True
if_using_td3 = True
if_using_sac = True

### Testing A2C

In [74]:
from stable_baselines3 import A2C, DDPG, PPO, SAC, TD3

trained_a2c = A2C.load("trained_models/agent_a2c") if if_using_a2c else None

In [75]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 30, State Space: 301


In [76]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [77]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

In [78]:
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_a2c, 
    environment = e_trade_gym) if if_using_a2c else (None, None)

hit end!


In [79]:
df_account_value_a2c.tail()

,date,account_value
699,2023-04-24,202771.734587
700,2023-04-25,202693.020278
701,2023-04-26,203183.409621
702,2023-04-27,204421.092497
703,2023-04-28,205911.574217


## Testing DDPG

In [68]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 30, State Space: 301


In [69]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [70]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

In [71]:
trained_ddpg = DDPG.load("trained_models/agent_ddpg") if if_using_ddpg else None

In [72]:
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_ddpg, 
    environment = e_trade_gym) if if_using_ddpg else (None, None)

hit end!


In [73]:
df_account_value_ddpg.tail()

,date,account_value
699,2023-04-24,198245.640598
700,2023-04-25,199362.430637
701,2023-04-26,200965.543185
702,2023-04-27,202362.565402
703,2023-04-28,204779.041232


## Testing PPO

In [53]:
stock_dimension = len(trade.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 30, State Space: 301


In [63]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 100000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}

In [64]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70, **env_kwargs)
env_trade, obs_trade = e_trade_gym.get_sb_env()

In [65]:
trained_ppo = PPO.load("trained_models/agent_ppo") if if_using_ppo else None

In [66]:
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_ppo, 
    environment = e_trade_gym) if if_using_ppo else (None, None)

hit end!


In [67]:
df_account_value_ppo.tail()

,date,account_value
699,2023-04-24,178891.439140
700,2023-04-25,180791.344291
701,2023-04-26,179884.571159
702,2023-04-27,181794.349540
703,2023-04-28,183229.323234


## Testing TD3

In [ ]:
#retrieve from github

In [ ]:
trained_td3 = agent.train_model(model=model_td3, 
                             tb_log_name='td3',
                             total_timesteps=50000) if if_using_td3 else None

In [ ]:
trained_td3.save(TRAINED_MODEL_DIR + "/agent_td3") if if_using_td3 else None

In [ ]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym) if if_using_td3 else (None, None)

In [ ]:
df_account_value_td3.tail()

In [ ]:
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_td3, 
    environment = e_trade_gym) if if_using_td3 else (None, None)

In [ ]:
df_account_value_td3.tail()

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

In [ ]:
trained_sac = agent.train_model(model=model_sac, 
                             tb_log_name='sac',
                             total_timesteps=70000) if if_using_sac else None

In [ ]:
trained_sac.save(TRAINED_MODEL_DIR + "/agent_sac") if if_using_sac else None

In [ ]:
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_sac, 
    environment = e_trade_gym) if if_using_sac else (None, None)

In [ ]:
df_account_value_sac.tail()